# Step 3: Normalisation
We have previously discussed the principles behind non-linear registration on this course. As a quick review, the reasons for normalising data to a standard space relate to issues of anatomical variability and the problems this poses for analysing groups of scans. In brief, the statistical analysis of fMRI data is performed in voxel-space. As such, it is necessary for all the images to be the same dimensions *and* for the same voxel coordinates to correspond to the same anatomical locations. Given how substantial anatomical variability can be, this will never be the case for raw fMRI data. As such, normalisation becomes a necessary step in order to bring all the images into the same coordinate space. However, given the complexity of the anatomical differences across subjects, simple affine registration approaches will not be suitable. Instead, we need to use *non-linear* methods to provide the best correspondance between the data and the standard space template.

## Direct Normalisation to a Template
The most basic form of spatial normalisation involves the direct registration of an image to an MNI template. This can be performed as either a *one-step* or *two-step* procedure. 

### One-step Normalisation
In a one-step approach, the image is directly registered with a matching template in MNI space. For instance, a T1 template would be used when normalising an antomical image and an EPI template would be used when normalising a BOLD image. This is shown in {numref}`one-step-norm-fig`. 

```{figure} images/one-step.png
---
width: 800px
name: one-step-norm-fig
---
Illustration of one-step normalisation.
```

Although this approach can work well for anatomical images, the poor resolution of the fMRI data, coupled with the limited contrast of the template, means that there is minimal anatomical information that can be used to improve the alignment. As such, most direct normalisation approaches rely on a *two-step* approach instead. 

### Two-step Normalisation
In a two-step approach, an anatomical image from the same subject is directly registered to a T1 template and then the resultant transformation is applied to the functional data. In theory, this should result in much more accurate normalisation, as the high-resolution anatomical detail from the T1 can be used to derive a much more detailed transformation which can then be applied to the low-resolution functional images. The only proviso is that the functional and anatomical data must be well-aligned before this transformation is estimated, otherwise it will not be accurately applied to the fMRI images. An example of two-step normalisation is shown in {numref}`two-step-norm-fig`.

```{figure} images/two-step-alt.png
---
width: 800px
name: two-step-norm-fig
---
Illustration of two-step normalisation.
```

### Direct Normalisation in SPM

For many years, direct normalisation was the main approach implemented in SPM. This can be seen in the `Old Normalise` tools available under `SPM > Tools > Old Normalise` within the batch system. The MNI templates used for this approach are also still available within the SPM folder under `spm12 > toolbox > OldNorm`, with options for normalising T1, T2, EPI, PET, SPECT and PD (proton density) images. However, in 2005, [Ashburner & Friston](https://pubmed.ncbi.nlm.nih.gov/15955494/) developed a new approach to normalisation known as *unified segmentation*. This is now the default method for normalising fMRI images in SPM, as we will explore below.

## Unified Segmentation
In the unified segmentation approach, the generation of normalisation parameters is part of the same algorithm used to segment an anatomical image into the different tissue classes. As such, this is a form of *two-step* normalisation, but one where the generation of the deformation field is part of the segmentation process, rather than through direct registration to an MNI template. You can read more about how this works in the advanced drop-down below.

:::{admonition} Advanced: The Unified Segmentation Algorithm
:class: dropdown
### Gaussian Mixture Models for Segmentation
To understand how unified segmentation works, we need to first understand how tissue segmentation is performed. Given that different tissue types are fairly easy to discern using a standard T1-weighted structural scan, it stands to reason that the voxel intensities should provide some information about the tissue classes. For instance, white matter is generally brighter than grey matter and CSF tends to be the darkest. We can get a sense of this by plotting the probability distributions for the intensity values of different manually-defined tissue types, as shown in {numref}`seg-tissues-fig`.

```{figure} images/seg-tissues.png
---
width: 500px
name: seg-tissues-fig
---
Example of the different intensity distribution for different tissue classes from a T1-weight anatomical image.
```

Most notable here is both the separation and overlap between the different distributions. If the distributions were entirely separate then we could easily classify the tissues based on intensity alone. Unfortunately, the degree of overlap between these distribution causes problems in terms of deciding whether a voxel is one tissue type or another. Putting this issue to one side for now, we can model the probability distributions for different tissues in an image by using a *Gaussian Mixture Model*. This is illustrated in {numref}`gauss-mix-fig`, where the yellow bars represent the histogram of image intensities and the black line represents the overall shape of the distribution. The blue lines demonstrate how this overall shape can be approximated by mixing together Gaussian distributions with different means and variances. This is very similar to the idea of recreating a signal using a linear combination of sines and cosines. As such, this approach can be thought of as defining a Gaussian *basis set* for the image intensities.

```{figure} images/gauss-mix.png
---
width: 600px
name: gauss-mix-fig
---
Illustration of a Gaussian mixture model.
```

After estimating the shapes of all the Gaussian distributions, we could then assess the probability that a voxel belongs to a tissue class by calculating the probability of its intensity value across all the distributions. Whichever distribution gives the highest probability represents the most likely tissue class. However, as indicated earlier, one of the issues is that there is a degree of overlap between these distributions. This means that certain intensity values will have similar probabilities for multiple tissue types. This speaks to a more general issue known as the *partial volume effect*, where a voxel can contain multiple tissue types. As such, additional information is needed in order to make a decision about the tissues classification. This additional information comes in the form of *tissue probability maps* (TPMs). These are images that contain probabilities indicating how likely each voxel in the brain is to be a particular tissue type. These probabilities can be used to compliment the intensity information from the mixture model.

### Tissue Probability Maps (TPMs)
In terms of normalisation, the most important element of the unified segmentation algorithm is the TPMs. Examples of the TPMs included with SPM are given in {numref}`tpm-fig` for grey matter, white matter, CSF and air/background. These images are derived from the [ICBM Probabilistic Atlases](http://www.bmap.ucla.edu/portfolio/atlases/ICBM_Probabilistic_Atlases/), which are built from hundreds of scans of young healthy adults. As these scans combine information from across multiple individuals, they are defined in MNI space. As such, in order to use the TPMs to make decisions about an individual subject's anatomy, the TPMs must be warped from MNI space into subject-space. As such, a deformation field that warps the TPMs from MNI-space to subject-space is created during the process of segmentation. Inverting this field provides the transformation from subject-space to MNI space that can be applied to normalise the functional data.

```{figure} images/tpm.png
---
width: 400px
name: tpm-fig
---
Example of the tissue probability maps (TPMs) for grey matter, white matter, CSF and air.
```
:::

## Anatomical and Functional Registration
Irrespective of whether we are using a direct normalisation approach or the unified segmentation approach, two-step normalisation depends upon accurate registration between the functional and anatomical data. If this is not the case, then the deformation field will warp the wrong parts of the functional image and the transformation will not work. This is demonstrated in {numref}`anat-func-reg-fig`. As such, the first step in normalisation is to register the structural and functional images together in world-space. Although this could be achieved in a variety of ways, the most efficient is to move the anatomical data so that it is aligned with the functional data. In terms of a reference, a single volume of the functional series could be selected, or an *average* volume after motion correction. In theory, the latter should be the most accurate representation of the entire fMRI series. As such, in SPM, it is usual to save the mean functional volume after realignment to use as the reference image for registration of the anatomical scan prior to normalisation.

```{figure} images/anat-func-reg.png
---
width: 600px
name: anat-func-reg-fig
---
Example of how misregistration between an anatomical and functional scan prior to two-step normalisation (*top*) will cause the warping parameters to be applied incorrectly to the functional data (*bottom*).
```

## Normalising Functional Data in SPM

Now that we have discussed the theory of spatial normalisation, we can examine how to perform this step using SPM. The video below will demonstrate how to normalise the realigned and slice timing corrected functional data, using the SPM graphical interface.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=5f260a9e-981f-4849-95a1-0eb0b88e44b5&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="normalisation.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>

:::{admonition} Advanced: Normalisation Using the Segment Module
:class: dropdown
Given that SPM uses the unified segmentation algorithm to perform normalisation, it is also possible to generate the deformation field for normalisation through the `Segment` module, rather than the `Normalisation` module. Doing so provides more control over the segmentation options and can be useful to tweak the normalisation beyond those parameters made available through `Normalise: Estimate`. The process of doing this is demonstrated in the video below.

<div style="max-width: 1280px"><div style="position: relative; padding-bottom: 56.25%; height: 0; overflow: hidden;"><iframe src="https://livemanchesterac.sharepoint.com/sites/UOM-FBMH-MSc-NCCN/_layouts/15/embed.aspx?UniqueId=557ea6a4-7b54-4682-8118-ffc08ed0cc54&embed=%7B%22hvm%22%3Atrue%2C%22ust%22%3Atrue%7D&referrer=StreamWebApp&referrerScenario=EmbedDialog.Create" width="1280" height="720" frameborder="0" scrolling="no" allowfullscreen title="normalisation-segment.mov" style="border:none; position: absolute; top: 0; left: 0; right: 0; bottom: 0; height: 100%; max-width: 100%;"></iframe></div></div>

:::